Reference: https://github.com/YuandZhang/Dongchedi 

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import csv
import requests # This command allows us to fetch URLs
from lxml import etree
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas

In [2]:
class Config:
    stopwords_file = './hit_stopwords.txt'

In [4]:
from lxml import etree #etree是基于ElementTree API的一种XML解析方式
import unicodedata
import html
import re
import jieba
#from utils.Configuration import Config
import base64
def remove_html(content,isdecode=None):#isdecode 判断是否先需要解码,需要解码为1
    # 移除html标签
    # &#x等编码问题
    if isdecode==1:
        content=base64.b64decode(content).decode(encoding='utf-8')#内容解码
    content = html.unescape(content)
    content = unicodedata.normalize('NFKD', content)
    # 将html的换行替换成字符换行
    selector = etree.HTML(content)
    str_list = selector.xpath('//text()')
    text = ''.join(str_list)
    # text = clean(text)#数据清理
    text = clean(text)  # 去除英文和标点
    text=remove_stopwords(text)

    return text

def clean(text):
    text = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return text

def remove_stopwords(text):
    stopwords = [i.strip() for i in open(Config.stopwords_file,encoding='utf-8').readlines()]
    text_depart=jieba.cut(text.strip())
    outstr=''
    for word in text_depart:
        if word not in stopwords:
            if word !='\t':
                outstr+=word
                outstr+=" "
    return outstr

def seg_tail_split(str1, sep=r":|,|，|。|n|！|!|：|'\'"):  # 分隔符可为多样的正则表达式
    # 保留分割符号，置于句尾，比如标点符号
    wlist = re.split(sep, str1)
    seg_word = re.findall(sep, str1)
    seg_word.extend(" ")  # 末尾插入一个空字符串，以保持长度和切割成分相同
    wlist = [x + y for x, y in zip(wlist, seg_word)]  # 顺序可根据需求调换
    return wlist

def ListCharReplace(ls,o,n):
    #x:需要的列表
    #o:原来的字符
    #n:现在的字符
    b = list(ls[0])
    rep = [n if x == o else x for x in b]
    s =["".join(rep)]
    return s

In [5]:
#从csv里获取车型id和名称
def getcarid():
    caridls=[]
    carnamels=[]
    with open('data_dongchedi.csv',encoding='utf-8') as cf:
        lines=cf.readlines()
        for line in lines:
            carid, carname=line.split(',')[0],line.split(',')[1]
            caridls.append(carid)
            carnamels.append(carname)
        return caridls,carnamels

In [6]:
#获取车辆id对应的口碑网页链接
def getUrl(carid):
    #精华评价
    url = 'https://www.dongchedi.com/auto/series/score/{}-x-S0-x-x-x'.format(carid)
    return url

In [7]:
def getRoot(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.80 Safari/537.36 Edg/86.0.622.43'}
    r=requests.get(url,headers=headers)
    # etree=html.etree
    root=etree.HTML(r.text)
    return root

In [39]:
#获取评论
def getComment(root,url,carid,carname):
    commentls=[]
    _url=url+'-1'
    root=getRoot(_url)        
    for comment in range(1,15): #默认每页只有14条数据，但是这个需要修改
        CommentRoot=root.xpath('//*[@id="__next"]/div[1]/div[2]/div[3]/section/section[1]/article[{}]/section/p/text()'.format(str(comment))) 
        commentls.append(CommentRoot)
    saveComment(commentls,carid,carname,'csv')

In [60]:
#获取评论-加上翻页
def getComment(root,url,carid,carname):
    #获取页数
    pages=root.xpath('//*[@id="__next"]/div[1]/div[2]/div[3]/section/section[1]/div[2]/ul/li[7]/a/span/text()')
    pages=int(pages[0])
    print(pages)
    
    commentls=[]

    for p in range(1,pages+1):
        _url=url+'-{}'.format(str(p))
        root=getRoot(_url)        

        for comment in range(1,15): #默认每页只有14条数据，但是这个需要修改
            CommentRoot=root.xpath('//*[@id="__next"]/div[1]/div[2]/div[3]/section/section[1]/article[{}]/section/p/text()'.format(str(comment))) 
            commentls.append(CommentRoot)
    return commentls
    saveComment(commentls,carid,carname,'csv')

In [61]:
caridls,carnamels=getcarid()
carid=caridls[1]
carname=carnamels[1]#当前爬取的车型
url = getUrl(carid)
root = getRoot(url)
getComment(root, url,carid,carname)

46


[['这次最后一次更新，9月提车，现在3月初，这次是首保换机油机滤后的体会，主要几个问题有结论了，坐标上海。开门见山：之前的机油没有问题，的确出厂加多了，我那批首检都抽了，后面都没增多的了。机油带汽油味店里说没问题就这个味，一开始不相信，首保后信了。\n1. 机油增多乳化之前是误会了，实际没有，之前首检有发现机油超过上限，4s抽到上刻度线不到一丢丢，这次换机油发现没有增多，和12月份一致，之前有说机油有汽油味，今天在4s店盯在车间，闻了拆封新机油0w-16，这款机油就这个味道，原厂机油就是带味道像汽油味。现在上海如春，以后应该没0下了，打消了顾虑顺利过冬，换好油水重新开始。\n\n2. 6个月开下来很满意，最满意是油耗，上海没有太多零下，最低晚上-3中午6度的样子，这最冷的时候我单程上班17km油耗7.5-8个油！现在这种10-20度之间的天气大概5.5-6.5之间，开驾驶辅助电脑跟车定速就偏低，自己踩就偏高，再踩也没过7。油耗过程大概发动后开始3km综合是10-15左右，到10km下到8左右，单程越长后面越来越靠近5-6的样子。\n\n3. 关于噪音，本人车子顶配，确实没有感觉吵，急加速因为没有升档转速变化重新来过的那个感觉。所以急加速声音持续变大是明显一点，但是开这种车，急加速毕竟少，总得来说还是汉兰达比较安静，尤其是有电的时候匀速巡航。顺带一提，6个月体验下来汉兰达底盘质感真的很棒，滤震啥的和我同事的老款5系感觉差不多。\n\n4. JBL音响比预想要好，系统自带QQ音乐可能线上播放加载的音质不行，所以感觉一般。实测手机储存着的和网易云预载的10+m那种品质的音乐差别很大。本人老家伙平时就周杰伦比较多，还有点抖音土味甜歌带动次打次那种。听起来感觉还是很不错。\n\n5. 异响啥的没有发生，雨刮挡板会动存在，4s说心里不舒服可以免费换，反正也没声音没震动没事不想去店里浪费时间下次保养再说吧\n\n最后一提，3个驾驶模式目前仅感觉油门灵敏度和放油滑行感觉有区别。\n\n比较遗憾是s档那几个动能回收档没有试过区别，日常就正常模式一直在开，eco模式头几个月一直用，以为能省很多油，后来发现也没省多少，带来油门不灵敏开起来别扭后来就一直没用了，运动档不喜欢仪表变红也一直不用。\n\n有一个问题，有一次下医院地库遇到堵车，堵了蛮久。全液晶仪表背景色有一次变黄，不是正常的蓝

In [38]:
def saveComment(commentls,carid,carname,filetype):
    if filetype=='csv':
        for i in range(0, len(commentls)):
            with open('data_carcomment.csv','a',encoding='utf-8',newline='') as dc:
                rep_comment=ListCharReplace(commentls[i],'{','(')
                dcwriter = csv.writer(dc, delimiter='{')
                dcwriter.writerow([carid,carname,commentls[i]])
    elif filetype=='txt':
        for i in range(0, len(commentls)):
            with open('data_carcomment.txt', mode='a', encoding='utf-8', newline='') as dc:
                dc.writelines(carid+'||||'+carname+'||||'+commentls[i])

In [41]:
def TidyComment():
    originfile='data_carcomment.csv'
    tidyfile='data_carcomment_cleaned.csv'
    textid=1
    with open(originfile, mode='r', encoding='utf-8', newline='')as ofile:
        with open(tidyfile,mode='a',encoding='utf-8')as tfile:
            lines=ofile.readlines()
            for line in lines:
                sen_ls=[]
                senid=1
                carid=line.split('{')[0]
                carname=line.split('{')[1]
                carcomment=line.split('{')[2][1:-2]
#分割文本-去停用词-

                split_comment_ls=seg_tail_split(carcomment)#分割成句子列表

                print('正在写入{}'.format(str(textid)))
                for sen in split_comment_ls:
                    sen=remove_stopwords(sen)
                    if len(sen)!=0:
                        sen_ls.append(sen)#生成新的句子列表sen_ls
                sen_len=len(sen_ls)#子句列表长度
                tfile.writelines(str(textid) + str(sen_len)+'\n')  # 写入文本序号,子句个数
                tfile.writelines(carid + ',' + carname + '\n')  # 写入车ID，车辆名称
                for _sen in sen_ls:
                    tfile.writelines(str(senid)+','+_sen+'\n')
                    senid=senid+1
                textid=textid+1

In [ ]:
if __name__ == '__main__':
    # Spider()
    TidyComment()